In [ ]:
import rasterio
import rasterio.plot
import pyproj
import matplotlib
import matplotlib.pyplot as plt
from osgeo import gdal
from osgeo import osr
import math
from rasterio.plot import show
import fiona
import rasterio
import rasterio.mask
import numpy
import sys
numpy.set_printoptions(threshold=sys.maxsize)
import os
import re
import statistics

In [ ]:
folder=r'Location for SPI rasters'
print(os.listdir(folder))

In [ ]:
#Changing directory to load files from and save output in
os.chdir(r'Location for SPI rasters')
#Checking if gdal library is loaded properly- sometimes it doesnt load properly
!gdalinfo -nomd spi_03_Gamma_X1981_01_water_masked.tif

In [ ]:
#making a list of all raster files
spi_names=[]
for file_name in os.listdir(folder):
    spi_names.append(file_name)

In [ ]:
#load one image to get various values needed later
img1=gdal.Open(r'Location for default image for Mask')
col=img1.RasterXSize
rows=img1.RasterYSize
driver=img1.GetDriver()
img1_val=img1.GetRasterBand(1).ReadAsArray().flatten()

In [ ]:
#mask creation to remove values outside of boundary in loaded tiff files values later
mask_in=[]
mask_out=[]
for i in range(0,len(img1_val)):
    if img1_val[i]==0:# replace with default value outside of boundary
        mask_out.append(i)
    else:
        mask_in.append(i)

In [ ]:
print(len(mask_in),len(mask_out),len(img1_val))

# Functions for Monitoring

In [ ]:
def masking_1(arr):
    masked_list=[]
    for i in mask_in:
        masked_list.append(arr[i])
    return masked_list

In [ ]:
def classification_1(abc):
    reslt=[]
    for i in range(0,len(abc)):
        if abc[i]<-0.5:#change value based on drought severity
            reslt.append(0)
        else:
            reslt.append(1)
    return reslt

In [ ]:
def rle_2(val_list):
    rle_output=[]
    i=0
    while (i <= len(val_list)-1):
        count=1
        ch=val_list[i]
        j=i
        while (j < len(val_list)-1):
            if (val_list[j] == val_list[j+1]):
                count=count+1
                j=j+1
            else:
                break
        rle_output.append(ch)
        rle_output.append(count)
        i=j+1
    return rle_output

In [ ]:
def monitoring(abc):
    result=[]
    freq=0
    month_count=0
    start_year=[]
    start_month=[]
    end_year=[]
    end_month=[]
    duration=[]
    for k in range(0,len(abc),2):
        if abc[k]==0 and abc[k+1]>=3:
            duration.append(abc[k+1])
            freq+=1
            sy=month_count//12
            sm=month_count%12
            sm=sm+1
            start_year.append(sy)
            start_month.append(sm)
            month_count=month_count+abc[k+1]
            em=sm+abc[k+1]-2
            em=em%12
            em=em+1
            end_month.append(em)
            ey=month_count//12
            end_year.append(ey)
        else:
            month_count=month_count+abc[k+1]
    result.append(freq)
    if freq==0:
        start_month.append(-1)
        end_month.append(-1)
        duration.append(-1)
    result.append(statistics.mode(start_month))
    result.append(statistics.mode(end_month))
    result.append(statistics.mean(duration))
    result.append(numpy.max(duration))
    result.append(start_year)
    return result

# Monitoring

In [ ]:
spi_test_val_2=[]
for i in range(2,len(spi_names)):#Change the starting index based on timescale- SPI1=0, SPI3=2, SPI6=5, SPI9=8 
    img=gdal.Open(spi_names[i])
    val=img.GetRasterBand(1).ReadAsArray().flatten()
    val=masking_1(val)
    val=classification_1(val)
    val=numpy.array(val)
    spi_test_val_2.append(val)
print(len(spi_test_val_2))

In [ ]:
pixel_val_i_full=[]
for i in spi_test_val_2:
    pixel_val_i_480=[]
    for j in range(0,len(i)):
        pixel_val_i_480.append(i[j])
    pixel_val_i_full.append(pixel_val_i_480)

In [ ]:
valid_frequency_count=[]
valid_mode_onset_month=[]
valid_mode_cessation_month=[]
valid_mean_duration=[]
valid_max_duration=[]
valid_start_years=[]
for i in range(0,len(mask_in)):
    pixel_val_i_calc=[]
    for j in pixel_val_i_full:
        pixel_val_i_calc.append(j[i])
    pixel_val_i_calc_rle=rle_2(pixel_val_i_calc)
    print("Results for pixel "+ str(i+1))
    result=monitoring(pixel_val_i_calc_rle)
    valid_frequency_count.append(result[0])
    valid_mode_onset_month.append(result[1])
    valid_mode_cessation_month.append(result[2])
    valid_mean_duration.append(result[3])
    valid_max_duration.append(result[4])
    valid_start_years.append(result[5])

In [ ]:
# Monitoring-1 Image Matrix Creation
frequency_values=[-9999]*12649
mode_onset_month_values=[-9999]*12649
mode_cessation_month_values=[-9999]*12649
mean_duration_values=[-9999]*12649
max_duration_values=[-9999]*12649
j=0
for i in mask_in:
    frequency_values[i]=valid_frequency_count[j]
    mode_onset_month_values[i]=valid_mode_onset_month[j]
    mode_cessation_month_values[i]=valid_mode_cessation_month[j]
    mean_duration_values[i]=valid_mean_duration[j]
    max_duration_values[i]=valid_max_duration[j]
    j=j+1

In [ ]:
# Frequncy Image
frequency_values=numpy.array(frequency_values)
frequencyMatrix=frequency_values.reshape(rows,col)
raster_frequency=driver.Create("Frequency_of_Drought.tif",col,rows,1,gdal.GDT_Float32)
raster_frequency.SetGeoTransform(img1.GetGeoTransform())
raster_frequency.SetProjection(img1.GetProjection())
raster_frequency.GetRasterBand(1).WriteArray(frequencyMatrix)
del raster_frequency

In [ ]:
# Mode Onset Month Image
mode_onset_month_values=numpy.array(mode_onset_month_values)
mode_monthMatrix=mode_onset_month_values.reshape(rows,col)
raster_mode_month=driver.Create("Mode_of_Onset_Month_of_Drought.tif",col,rows,1,gdal.GDT_Float32)
raster_mode_month.SetGeoTransform(img1.GetGeoTransform())
raster_mode_month.SetProjection(img1.GetProjection())
raster_mode_month.GetRasterBand(1).WriteArray(mode_monthMatrix)
del raster_mode_month

In [ ]:
# Mode Cessation Month Image
mode_cessation_month_values=numpy.array(mode_cessation_month_values)
mode_month_endMatrix=mode_cessation_month_values.reshape(rows,col)
raster_mode_month_end=driver.Create("Mode_of_Cessation_Month_of_Drought_End.tif",col,rows,1,gdal.GDT_Float32)
raster_mode_month_end.SetGeoTransform(img1.GetGeoTransform())
raster_mode_month_end.SetProjection(img1.GetProjection())
raster_mode_month_end.GetRasterBand(1).WriteArray(mode_month_endMatrix)
del raster_mode_month_end

In [ ]:
# Mean Duration Image
mean_duration_values=numpy.array(mean_duration_values)
mean_durationMatrix=mean_duration_values.reshape(rows,col)
raster_mean_duration=driver.Create("Mean_Duration.tif",col,rows,1,gdal.GDT_Float32)
raster_mean_duration.SetGeoTransform(img1.GetGeoTransform())
raster_mean_duration.SetProjection(img1.GetProjection())
raster_mean_duration.GetRasterBand(1).WriteArray(mean_durationMatrix)
del raster_mean_duration

In [ ]:
# Max Duration Image
max_duration_values=numpy.array(max_duration_values)
max_durationMatrix=max_duration_values.reshape(rows,col)
raster_max_duration=driver.Create("Maximum_Duration.tif",col,rows,1,gdal.GDT_Float32)
raster_max_duration.SetGeoTransform(img1.GetGeoTransform())
raster_max_duration.SetProjection(img1.GetProjection())
raster_max_duration.GetRasterBand(1).WriteArray(max_durationMatrix)
del raster_max_duration

# For Year Wise Drought Verification

In [ ]:
for i in valid_start_years:
    for j in range(0,len(i)):
        i[j]=i[j]+1981

In [ ]:
def Remove(duplicate):
    final_list = []
    for num in duplicate:
        if num not in final_list:
            final_list.append(num)
    return final_list

In [ ]:
drought_occurence_total=[]
for i in valid_start_years:
    drought_occurence_pixel=Remove(i)
    drought_occurence_total.append(drought_occurence_pixel)
print(drought_occurence_total)

In [ ]:
drought_map=[]
for i in range(1981,2021):
    y=[]
    for j in drought_occurence_total:
        if i in j:
            y.append(1)
        else:
            y.append(0)
    drought_map.append(y)

# Drought Image for Each Year

In [ ]:
l=1981
for i in range(0,40):
    drought_pixels=[-9999]*12649
    k=0
    for j in mask_in:
        drought_pixels[j]=drought_map[i][k]
        k=k+1
    print(drought_pixels)
# #image Creation for each year
    drought_pixels_2=numpy.array(drought_pixels)
    droughtMatrix=drought_pixels_2.reshape(rows,col)
    raster_drought=driver.Create("Drought_in_"+str(l)+".tif",col,rows,1,gdal.GDT_Float32)
    l=l+1
    raster_drought.SetGeoTransform(img1.GetGeoTransform())
    raster_drought.SetProjection(img1.GetProjection())
    raster_drought.GetRasterBand(1).WriteArray(droughtMatrix)
    del raster_drought

# Monitoring Phase 2- Intensity(Average and Maximum)

In [ ]:
def monitoring_2(abc):
    result=[]
    freq=0
    month_count=0
    start_month=[]
    end_month=[]
    for k in range(0,len(abc),2):
        if abc[k]==0 and abc[k+1]>=3:
            freq+=1
            start_month.append(month_count)
            month_count=month_count+abc[k+1]
            end_month.append(month_count)
        else:
            month_count=month_count+abc[k+1]
    result.append(freq)
    if freq==0:
        start_month.append(-1)
        end_month.append(-1)
    result.append(start_month)
    result.append(end_month)
    return result

In [ ]:
#Reloading rasters
spi_test_val_3=[]
for i in range(2,len(spi_names)):#Change the starting index based on timescale- SPI1=0, SPI3=2, SPI6=5, SPI9=8 
    img=gdal.Open(spi_names[i])
    val=img.GetRasterBand(1).ReadAsArray().flatten()
    val=masking_1(val)
    spi_test_val_3.append(val)
print(len(spi_test_val_3))

In [ ]:
spi_test_val_4=[]
for i in range(0,len(mask_in)):
    pixel_val=[]
    for j in spi_test_val_3:
        pixel_val.append(j[i])
    spi_test_val_4.append(pixel_val)
print(len(spi_test_val_4))

In [ ]:
spi_test_val_5=[]
for i in range(0,len(mask_in)):
    tempo_val=[]
    for j in spi_test_val_3:
        tempo_val.append(j[i])
    tempo_val=classification_1(tempo_val)
    spi_test_val_5.append(tempo_val)

In [ ]:
spi_test_val_5_rle=[]
for i in spi_test_val_5:
    spi_test_val_5_rle.append(rle_2(i))

In [ ]:
result_check=[]
for i in spi_test_val_5_rle:
    result_check.append(monitoring_2(i))

In [ ]:
avg_intensity_values=[]
max_intensity_values=[]
for i in range(0,len(result_check)):
    intensity_values=[]
    if result_check[i][0]==0:
        intensity_values.append(0)
    for j in range(0,result_check[i][0]):
        a=result_check[i][1][j]
        b=result_check[i][2][j]
        int_values=statistics.mean(spi_test_val_4[i][a:b])
        intensity_values.append(int_values)
    avg_intensity_values.append(statistics.mean(intensity_values))
    max_intensity_values.append(numpy.min(intensity_values))

In [ ]:
mean_severity=[]
for i in spi_test_val_4:
    mean_severity.append(statistics.mean(i))

In [ ]:
#Image Matrix Creation
average_intensity_values=[-9999]*12649
maximum_intensity_values=[-9999]*12649
mean_severity_values=[-9999]*12649
j=0
for i in mask_in:
    average_intensity_values[i]=avg_intensity_values[j]
    maximum_intensity_values[i]=max_intensity_values[j]
    mean_severity_values[i]=mean_severity[j]
    j=j+1

In [ ]:
# Average Intensity Image
average_intensity_values=numpy.array(average_intensity_values)
average_intensityMatrix=average_intensity_values.reshape(rows,col)
raster_average_intensity=driver.Create("Average_Intensity.tif",col,rows,1,gdal.GDT_Float32)
raster_average_intensity.SetGeoTransform(img1.GetGeoTransform())
raster_average_intensity.SetProjection(img1.GetProjection())
raster_average_intensity.GetRasterBand(1).WriteArray(average_intensityMatrix)
del raster_average_intensity

In [ ]:
# Maximum Intensity Image
maximum_intensity_values=numpy.array(maximum_intensity_values)
max_intensityMatrix=maximum_intensity_values.reshape(rows,col)
raster_max_intensity=driver.Create("Maximum_Intensity.tif",col,rows,1,gdal.GDT_Float32)
raster_max_intensity.SetGeoTransform(img1.GetGeoTransform())
raster_max_intensity.SetProjection(img1.GetProjection())
raster_max_intensity.GetRasterBand(1).WriteArray(max_intensityMatrix)
del raster_max_intensity

In [ ]:
# Mean Severity Image
mean_severity_values=numpy.array(mean_severity_values)
mean_severityMatrix=mean_severity_values.reshape(rows,col)
raster_mean_severity=driver.Create("Mean_Severity.tif",col,rows,1,gdal.GDT_Float32)
raster_mean_severity.SetGeoTransform(img1.GetGeoTransform())
raster_mean_severity.SetProjection(img1.GetProjection())
raster_mean_severity.GetRasterBand(1).WriteArray(mean_severityMatrix)
del raster_mean_severity


# Top Ten Locations with Highest Value of each Characteristic

In [ ]:
#Function to find Maximum for Frequnecy and Duration
def top_ten_indices_max(lst):
    top_ten=[]
    while True:
        ind = [index for index, item in enumerate(lst) if item == max(lst)]
        for i in ind:
            top_ten.append(i)
            lst[i]=-9999
            if len(top_ten)==10:
                return top_ten

In [ ]:
#Function to find Maximum(actually Minimum) for Intensity and Severity
def top_ten_indices_min(lst):
    top_ten=[]
    while True:
        ind = [index for index, item in enumerate(lst) if item == min(lst)]
        for i in ind:
            top_ten.append(i)
            lst[i]=9999
            if len(top_ten)==10:
                return top_ten

In [ ]:
# Top Frequency Locations Map
f=valid_frequency_count.copy()
top_f=top_ten_indices_max(f)
q=[]
for i in top_f:
    q.append(mask_in[i])
ttf=[-9999]*len(img1_val)
for i in mask_in:
    ttf[i]=9999
for i in range(0,len(q)):
    a=q[i]
    b=top_f[i]
    ttf[a]=valid_frequency_count[b]
ttf=numpy.array(ttf)
ttfMatrix=ttf.reshape(rows,col)
raster_ttf=driver.Create("Top_ten_Frequency.tif",col,rows,1,gdal.GDT_Float32)
raster_ttf.SetGeoTransform(img1.GetGeoTransform())
raster_ttf.SetProjection(img1.GetProjection())
raster_ttf.GetRasterBand(1).WriteArray(ttfMatrix)
del raster_ttf

In [ ]:
# Top Mean Duration Locations Map
d=valid_mean_duration.copy()
top_d=top_ten_indices_max(d)
q=[]
for i in top_d:
    q.append(mask_in[i])
ttd=[-9999]*len(img1_val)
for i in mask_in:
    ttd[i]=9999
for i in range(0,len(q)):
    a=q[i]
    b=top_f[i]
    ttd[a]=valid_mean_duration[b]
ttd=numpy.array(ttd)
ttdMatrix=ttd.reshape(rows,col)
raster_ttd=driver.Create("Top_ten_mean_duration.tif",col,rows,1,gdal.GDT_Float32)
raster_ttd.SetGeoTransform(img1.GetGeoTransform())
raster_ttd.SetProjection(img1.GetProjection())
raster_ttd.GetRasterBand(1).WriteArray(ttdMatrix)
del raster_ttd

In [ ]:
# Top Maximum Duration Locations Map
d=valid_max_duration.copy()
top_d=top_ten_indices_max(d)
q=[]
for i in top_d:
    q.append(mask_in[i])
ttd=[-9999]*len(img1_val)
for i in mask_in:
    ttd[i]=9999
for i in range(0,len(q)):
    a=q[i]
    b=top_f[i]
    ttd[a]=valid_max_duration[b]
ttd=numpy.array(ttd)
ttdMatrix=ttd.reshape(rows,col)
raster_ttd=driver.Create("Top_ten_max_duration.tif",col,rows,1,gdal.GDT_Float32)
raster_ttd.SetGeoTransform(img1.GetGeoTransform())
raster_ttd.SetProjection(img1.GetProjection())
raster_ttd.GetRasterBand(1).WriteArray(ttdMatrix)
del raster_ttd

In [ ]:
# Top Mean Severity Locations Map
d=mean_severity.copy()
top_d=top_ten_indices_min(d)
q=[]
for i in top_d:
    q.append(mask_in[i])
ttd=[-9999]*len(img1_val)
for i in mask_in:
    ttd[i]=9999
for i in range(0,len(q)):
    a=q[i]
    b=top_f[i]
    ttd[a]=mean_severity[b]
ttd=numpy.array(ttd)
ttdMatrix=ttd.reshape(rows,col)
raster_ttd=driver.Create("Top_ten_mean severity.tif",col,rows,1,gdal.GDT_Float32)
raster_ttd.SetGeoTransform(img1.GetGeoTransform())
raster_ttd.SetProjection(img1.GetProjection())
raster_ttd.GetRasterBand(1).WriteArray(ttdMatrix)
del raster_ttd

In [ ]:
# Top maximum Intensity Locations Map
d=max_intensity_values.copy()
top_d=top_ten_indices_min(d)
q=[]
for i in top_d:
    q.append(mask_in[i])
ttd=[-9999]*len(img1_val)
for i in mask_in:
    ttd[i]=9999
for i in range(0,len(q)):
    a=q[i]
    b=top_f[i]
    ttd[a]=max_intensity_values[b]
ttd=numpy.array(ttd)
ttdMatrix=ttd.reshape(rows,col)
raster_ttd=driver.Create("Top_ten_maximum_intensity.tif",col,rows,1,gdal.GDT_Float32)
raster_ttd.SetGeoTransform(img1.GetGeoTransform())
raster_ttd.SetProjection(img1.GetProjection())
raster_ttd.GetRasterBand(1).WriteArray(ttdMatrix)
del raster_ttd

In [ ]:
# Top Mean Intensity Locations Map
d=avg_intensity_values.copy()
top_d=top_ten_indices_min(d)
q=[]
for i in top_d:
    q.append(mask_in[i])
ttd=[-9999]*len(img1_val)
for i in mask_in:
    ttd[i]=9999
for i in range(0,len(q)):
    a=q[i]
    b=top_f[i]
    ttd[a]=avg_intensity_values[b]
ttd=numpy.array(ttd)
ttdMatrix=ttd.reshape(rows,col)
raster_ttd=driver.Create("Top_ten_average_intensity.tif",col,rows,1,gdal.GDT_Float32)
raster_ttd.SetGeoTransform(img1.GetGeoTransform())
raster_ttd.SetProjection(img1.GetProjection())
raster_ttd.GetRasterBand(1).WriteArray(ttdMatrix)
del raster_ttd